# Import Libraries

In [186]:
# Pandas & Numpy 
import pandas as pd
import numpy as np

# Tensorflow
import tensorflow as tf
import tensorflow_recommenders as tfrs

tf.compat.v1.enable_eager_execution()

# Types
from typing import Dict, Text

# System
import os 
import pprint
import tempfile

# Loading & Reading Data

In [187]:
# Read original Data to collect all columns
original_df  = pd.read_csv('../../00_Data/Rec_Sys_precleaned.csv', index_col=0)

In [188]:
# Check Datatypes to identify column types that need to be changed
original_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245898 entries, 0 to 272403
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   InvoiceNo         245898 non-null  int64  
 1   StockCode         245898 non-null  object 
 2   Quantity          245898 non-null  int64  
 3   InvoiceDate       245898 non-null  object 
 4   DeliveryDate      245898 non-null  object 
 5   Discount%         245898 non-null  float64
 6   ShipMode          245898 non-null  object 
 7   ShippingCost      245898 non-null  float64
 8   CustomerID        245898 non-null  int64  
 9   Gender            245898 non-null  object 
 10  Age               245898 non-null  int64  
 11  Income            245898 non-null  object 
 12  Zipcode           245898 non-null  int64  
 13  Customer Segment  245898 non-null  object 
 14  Product Name      245898 non-null  object 
 15  Description       245898 non-null  object 
 16  Category          24

In [189]:
# CustomerID

In [190]:
# Convert CustomerID to String
original_df['CustomerID']  = original_df['CustomerID'].astype(str)
#original_df['Age'] = original_df['Age'].astype(str)


# Selection of Features

In [191]:
# First get the list of features
original_df.columns

Index(['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'DeliveryDate',
       'Discount%', 'ShipMode', 'ShippingCost', 'CustomerID', 'Gender', 'Age',
       'Income', 'Zipcode', 'Customer Segment', 'Product Name', 'Description',
       'Category', 'Brand', 'Unit Price', 'Num_word_text', 'Num_word_cat'],
      dtype='object')

In [192]:
# Select important Features
interactions_dict = original_df.groupby(['CustomerID','StockCode', 'Age', 'Gender', 'Product Name', 'Description']).agg({'CustomerID': 'count'})
interactions_dict.columns = ['purchased']
interactions_dict.reset_index(inplace=True)
# Set all purchases to 1 
interactions_dict['purchased'] = 1 

# Transform Table into a Dictionary, which will be fed into tensor slices
interactions_dict = {name: np.array(value) for name, value in interactions_dict.items()} 
interactions = tf.data.Dataset.from_tensor_slices(interactions_dict)

## map the features in interactions
interactions = interactions.map(lambda x: {
    'user_id' : x['CustomerID'], 
    'product_id' : x['StockCode'], 
    'purchased' : float(x['purchased']),
    'age' : x['Age'],
    'gender' : x['Gender'],
    'product_name' : x['Product Name'],
    'description' : x['Description']
})

In [193]:
# We do a similar step for items, where this is the reference table for items to be recommended 
items_dict = original_df[['StockCode']].drop_duplicates()
items_dict = {name: np.array(value) for name, value in items_dict.items()}
items = tf.data.Dataset.from_tensor_slices(items_dict)
items = items.map(lambda x: x['StockCode'])

In [194]:
# Prepare feature Vocabularies
unique_item_titles = np.unique(np.concatenate(list(interactions.batch(1000).map(lambda x: x['product_id']))))
unique_user_ids = np.unique(np.concatenate(list(interactions.batch(1_000).map(lambda x: x["user_id"]))))

2023-07-09 18:02:26.258676: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [172701]
	 [[{{node Placeholder/_2}}]]
2023-07-09 18:02:29.442939: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [172701]
	 [[{{node Placeholder/_5}}]]


# Feature Preparation

## Categorical Features

### Product ID 

In [195]:
# Define Vocabulary for Product ID 
product_id_lookup = tf.keras.layers.StringLookup()

# Fill the Vocabulary with Data
product_id_lookup.adapt(interactions.map(lambda x: x['product_id']))

2023-07-09 18:02:32.562853: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [172701]
	 [[{{node Placeholder/_0}}]]


In [196]:
# Define Embeddings for Product ID
product_id_embedding = tf.keras.layers.Embedding(
    input_dim=product_id_lookup.vocab_size(),
    output_dim=32
)

In [197]:
# Put embedding & lookup into a single layer
product_id_model = tf.keras.Sequential([product_id_lookup, product_id_embedding])

In [198]:
# Print a product ID for test
original_df['StockCode'][1]

'71053'

In [199]:
# Get the embedding of a Product 
product_id_model(['71053'])

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[ 0.01646421,  0.00617598, -0.02800252, -0.00169883, -0.00604776,
         0.00639004, -0.04687766,  0.02102312,  0.03963062, -0.00850894,
        -0.03849081, -0.02605684, -0.0282975 ,  0.01856469, -0.02912003,
         0.0474073 , -0.0327639 ,  0.01924804,  0.02751591,  0.00555352,
        -0.03492472, -0.03088921,  0.02236874,  0.02585094,  0.02079472,
         0.02555586,  0.01308027,  0.03799304, -0.00532647, -0.02995148,
         0.03740298,  0.01962529]], dtype=float32)>

### User ID

In [200]:
# Define Vocabulary for User ID 
user_id_lookup = tf.keras.layers.StringLookup()

# Fill the Vocabulary with Data
user_id_lookup.adapt(interactions.map(lambda x: x['user_id']))

2023-07-09 18:03:22.040780: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [172701]
	 [[{{node Placeholder/_1}}]]


In [201]:
# Define Embeddings for User ID
user_id_embedding = tf.keras.layers.Embedding(
    input_dim=user_id_lookup.vocab_size(),
    output_dim=32
)

In [202]:
# Put embedding & lookup into a single layer
user_id_model = tf.keras.Sequential([user_id_lookup, user_id_embedding])

In [203]:
# Test Embedding
user_id_model(['1'])

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-0.04689747, -0.01459377, -0.04676555,  0.0460869 ,  0.04572833,
        -0.03261002,  0.03727752,  0.04489819,  0.02560166,  0.03489121,
         0.0120306 , -0.03286077, -0.01389351,  0.02179532,  0.04140956,
         0.01095779, -0.00947567, -0.04009696,  0.01637847,  0.02494958,
         0.03687007,  0.02795488,  0.04721668,  0.01051784,  0.04936339,
         0.00926585, -0.02690964,  0.00142392, -0.00755669, -0.03264137,
         0.02260501, -0.03117102]], dtype=float32)>

### Gender

In [204]:
# Define Vocabulary
gender_lookup = tf.keras.layers.StringLookup()

# Fill Vocabulary with Data
gender_lookup.adapt(interactions.map(lambda x: x['gender']))

# Create Embeddings for Gender
gender_embedding = tf.keras.layers.Embedding(gender_lookup.vocab_size(), 32)

# Create model
gender_model = tf.keras.Sequential([gender_lookup, gender_embedding])

2023-07-09 18:04:12.222144: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [172701]
	 [[{{node Placeholder/_1}}]]


In [205]:
# Print a product ID for test
original_df['Gender'][1]

'female'

In [206]:
gender_model(['female'])

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-0.01443794,  0.01501573, -0.0068123 , -0.04261998, -0.04675772,
         0.02150499,  0.02387691, -0.00284144, -0.04419537,  0.00697934,
         0.023961  ,  0.00329473, -0.00708721, -0.00152894,  0.02587315,
        -0.01144733, -0.03962497,  0.01132978, -0.04253967,  0.0041736 ,
        -0.01695894,  0.01060377, -0.03989327, -0.0058979 ,  0.03171369,
        -0.04112272,  0.04255808, -0.03323274,  0.01077366, -0.01017828,
        -0.04035743, -0.02766221]], dtype=float32)>

## Numeric Features

### Age

In [207]:
# Check Data Type of Age as Sanity Check
original_df.Age.info()

<class 'pandas.core.series.Series'>
Int64Index: 245898 entries, 0 to 272403
Series name: Age
Non-Null Count   Dtype
--------------   -----
245898 non-null  int64
dtypes: int64(1)
memory usage: 11.8 MB


In [208]:
# Standardization of Age

# Set up Object
age_normalization = tf.keras.layers.Normalization(
    axis=None
)

# Fill with Data
age_normalization.adapt(interactions.map(lambda x: x['age']).batch(1024))


2023-07-09 18:05:02.375232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [172701]
	 [[{{node Placeholder/_5}}]]


In [209]:
# Discretization of Age
max_age = interactions.map(lambda x: x['age']).reduce(
    tf.cast(0, tf.int64), tf.maximum).numpy().max()
min_age = interactions.map(lambda x: x['age']).reduce(
    np.int64(1e9), tf.minimum).numpy().min()

age_buckets = np.linspace(
    min_age, max_age, num=1000
) 

2023-07-09 18:05:05.735749: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [172701]
	 [[{{node Placeholder/_5}}]]
2023-07-09 18:05:13.322077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [172701]
	 [[{{node Placeholder/_5}}]]


## Text Features

### Product Name

In [210]:
# Vectorization for Product Name
product_text = tf.keras.layers.TextVectorization()
product_text.adapt(interactions.map(lambda x: x['product_name']))

2023-07-09 18:05:20.155472: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype string and shape [172701]
	 [[{{node Placeholder/_3}}]]


### Product Description

In [211]:
# Vectorization for Product Description
product_description = tf.keras.layers.TextVectorization()
product_description.adapt(interactions.map(lambda x: x['description']))

2023-07-09 18:06:16.646685: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype string and shape [172701]
	 [[{{node Placeholder/_4}}]]


# Model Definition

## Query Model

In [212]:
class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.user_id_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids)+1,32)
        ])
        
        self.gender_embedding = tf.keras.Sequential([
            gender_lookup,
            tf.keras.layers.Embedding(gender_lookup.vocab_size(),32)
        ])
        
        self.age_embeding = tf.keras.Sequential([
            tf.keras.layers.Discretization(age_buckets.tolist()),
            tf.keras.layers.Embedding(len(age_buckets) +1, 32)
        ])
        
    def call(self, inputs):
        return tf.concat([
            self.user_id_embedding(inputs['user_id']),
            self.gender_embedding(inputs['gender']),
            self.age_embeding(inputs['age']),
        ], axis=1)

## Candidate Model

In [213]:
class ItemModel(tf.keras.Model): 
    def __init__(self):
        super().__init__()
        
        max_tokens = 10_000
        # Product ID  
        self.product_id_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_item_titles,mask_token=None),
            tf.keras.layers.Embedding(len(unique_item_titles) + 1, 32)
        ])
        
        # Product Name 
        self.product_name_vectorizer = tf.keras.layers.TextVectorization(
            max_tokens=max_tokens
        )
        self.product_name_embedding = tf.keras.Sequential([
            self.product_name_vectorizer,
            tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
            tf.keras.layers.GlobalAveragePooling1D()
        ]) 
        self.product_name_vectorizer.adapt(items)
        
        # Product Description
        self.product_desc_vectorizer = tf.keras.layers.TextVectorization(
            max_tokens=max_tokens
        ) 
        self.product_desc_embedding = tf.keras.Sequential([
            self.product_desc_vectorizer,
            tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
            tf.keras.layers.GlobalAveragePooling1D()
        ])
        self.product_desc_vectorizer.adapt(items)
        
    def call(self, titels):
        return tf.concat([
            self.product_id_embedding(titels),
            self.product_name_embedding(titels),
            self.product_desc_embedding(titels)
        ], axis=1) 

## Combined Model

In [214]:
class RetailModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        self.query_model = tf.keras.Sequential([
            UserModel(),
            tf.keras.layers.Dense(32)
        ])
        self.candidate_model = tf.keras.Sequential([
            ItemModel(),
            tf.keras.layers.Dense(32)
        ]) 
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=items.batch(128).map(self.candidate_model),
            ),
        )
        
    def compute_loss(self, features, training=False):
        query_embeddings = self.query_model({
            "user_id" : features['user_id'],
            "gender" : features['gender'],
            "age" : features['age'],
        })
        item_embeddings = self.candidate_model({
            "product_id" : features['product_id'],
            "product_name" : features['product_name'],
            "product_description" : features['description'],
        })
        
        return self.task(query_embeddings, item_embeddings)

# Train Test Split

In [229]:
original_df.shape

(245898, 21)

In [232]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = interactions.shuffle(200_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(160_000)
test = shuffled.skip(160_000).take(40_000)

cached_train = train.shuffle(160_000).batch(2048)
cached_test = test.batch(4096).cache()

# Building the Model

In [233]:
try:
    del(model)
except:
    pass

model = RetailModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

#evaluate the model
model.evaluate(cached_test, return_dict=True)

Epoch 1/3


2023-07-09 18:28:56.595997: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [172701]
	 [[{{node Placeholder/_0}}]]
2023-07-09 18:28:56.596424: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [172701]
	 [[{{node Placeholder/_5}}]]


2023-07-09 18:29:00.561200: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2349]
	 [[{{node Placeholder/_0}}]]


 1/79 [..............................] - ETA: 5:33 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0029 - factorized_top_k/top_50_categorical_accuracy: 0.0146 - factorized_top_k/top_100_categorical_accuracy: 0.0352 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan

2023-07-09 18:29:00.906856: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype string
	 [[{{node Placeholder/_11}}]]


...

2/4 [==============>...............] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan 

2023-07-09 18:30:22.565628: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node Placeholder/_6}}]]


4/4 [==============================] - 5s 278ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan


2023-07-09 18:30:22.923420: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_15' with dtype int64
	 [[{{node Placeholder/_15}}]]


{'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.0,
 'factorized_top_k/top_50_categorical_accuracy': 0.0,
 'factorized_top_k/top_100_categorical_accuracy': 0.0,
 'loss': nan,
 'regularization_loss': 0,
 'total_loss': nan}

# Evaluate Model

In [226]:
train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

 1/30 [>.............................] - ETA: 1:21 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan

2023-07-09 18:16:59.071626: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype resource
	 [[{{node Placeholder/_9}}]]
2023-07-09 18:16:59.266033: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64
	 [[{{node Placeholder/_12}}]]


 2/30 [=>............................] - ETA: 6s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan  

2023-07-09 18:16:59.490570: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64
	 [[{{node Placeholder/_10}}]]


 3/30 [==>...........................] - ETA: 6s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan

...

3/5 [=================>............] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan

2023-07-09 18:17:06.619131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_24' with dtype float and shape [4096,32]
	 [[{{node Placeholder/_24}}]]


4/5 [=======================>......] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan

2023-07-09 18:17:06.951221: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype resource
	 [[{{node Placeholder/_9}}]]


5/5 [==============================] - 2s 352ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan
Top-100 accuracy (train): 0.00.
Top-100 accuracy (test): 0.00.


# Predictions

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model, k=10)

index.index_from_dataset(items.batch(100).map(lambda title: (title, model.candidate_model(title))))

2023-07-06 17:53:52.158526: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64
	 [[{{node Placeholder/_12}}]]


# Create Top K List per User

In [ ]:
# First we select all users in a list 
all_users = original_df['CustomerID'].unique().tolist()

In [ ]:
all_users[1]

'13047'

In [ ]:
# Create Data Frame 
# First Set up empty lists 
c_name = []
i_name = []

# Iterate through all users 
for u in all_users:
    for i in range(10):
        ranked_item = index(np.array([u]))[1][0][i].numpy()
        c_name.append(u)
        i_name.append(ranked_item)


# Create Dataframe         
df_predictions = pd.DataFrame({
    "CustomerID": c_name,
    "StockCode" : i_name
})

# Add Predictions Column
df_predictions['predictions'] = 1.0

In [ ]:
# Create Test Dataset
test_list = list(test.as_numpy_iterator())
df_test = pd.DataFrame(test_list, columns=['user_id', 'product_id', 'purchased'])
df_test['user_id'] = df_test['user_id'].str.decode('utf-8')
df_test['product_id'] = df_test['product_id'].str.decode('utf-8')
df_test = df_test.rename(columns={'user_id':'CustomerID', 'product_id':'StockCode'})


In [ ]:
# Ensure that the same items and users are in both sets 
# Find the set of unique items in both the train and test sets
prediction_items = set(df_predictions['CustomerID'].unique())
test_items = set(df_test['CustomerID'].unique())

# Find the intersection of the sets from step 1
common_items = prediction_items.intersection(test_items)

# Filter the train and test sets to include only the rows with item IDs that are in the intersection set
df_predictions = df_predictions[df_predictions['CustomerID'].isin(common_items)]
df_test = df_test[df_test['CustomerID'].isin(common_items)]

In [ ]:
# Send to csv for further assessment
df_predictions.to_csv('pred_hybrid.csv',index=False)
df_test.to_csv('test_hybrid.csv', index=False)